Initializing Libraries and Global Variables

In [21]:
import pickle
import pandas as pd
import psycopg2

In [22]:
path = r'D:\\Projects\\ipums\\garrard_test\\'
cbkname = r'ky_1870_cbk.pickle'
prefix = 'cn70_'
table_name = 'census_1870'
cbkfile = path + cbkname

Pickle functions

In [23]:
def load_pickle(filename, objectname):
        with open(path+filename, "rb") as infile:
            objectname = pickle.load(infile)
        print('Loaded from:', filename)

def save_pickle(filename, objectname):
    with open(path+filename, "wb") as outfile:
        pickle.dump(objectname, outfile)
    print('Saved to:',filename)

Loading the codebook and creating the column list

In [24]:

cbk = pd.read_pickle(cbkfile)
cbk.columns
x = cbk['length'].astype('str')
cbk['column_name'] = prefix+cbk['variable'].str.lower() + ' char('+x+')'
column_list = cbk['column_name'].tolist()
print('column_list:\n',column_list[0:10])

column_list:
 ['cn70_year char(4)', 'cn70_serial char(8)', 'cn70_numprec char(2)', 'cn70_subsamp char(2)', 'cn70_numperhh char(4)', 'cn70_hhtype char(1)', 'cn70_region char(2)', 'cn70_stateicp char(2)', 'cn70_statefip char(2)', 'cn70_countyicp char(4)']


Database Functions (psycopg2)

In [25]:
def open_database(database):
    conn = psycopg2.connect(dbname=database, user='postgres', password = '3701', host='localhost', port='5432')
    return conn

def create_cursor():
    cursor = conn.cursor()
    return cursor

def set_census_schema():
    cur.execute("SET SCHEMA 'census'")

def create_table(tablename, columns):
    print(len(column_list))
    columns_text = '(' + ', '.join(columns) + ');'
    cur.execute('DROP TABLE IF EXISTS ' + tablename + ';')
    cur.execute('CREATE TABLE ' + tablename + ' ' + columns_text)

def get_column_counts(tablename, columns):
    qry_cc = 'select count(*) from information_schema.columns where tablename =' + table_name + ';'
    col_ct_sql = cur.execute(qry_cc)
    col_ct_list = len(columns)+1
    print('columns in table:',col_ct_sql,'\n','columns in list:',col_ct_list)
    
def close_database():
    conn.commit() #insert rollback logic
    conn.close()
      

In [26]:
#testing sql strings
columns = column_list
columns_text = '(' + 'id serial PRIMARY KEY, ' + ', '.join(columns) + ');'
print(columns_text)
print ('DROP TABLE IF EXISTS ' + table_name + ';')
print('CREATE TABLE ' + table_name + ' ' + columns_text)
print('SELECT * FROM ' + table_name + ';')

(id serial PRIMARY KEY, cn70_year char(4), cn70_serial char(8), cn70_numprec char(2), cn70_subsamp char(2), cn70_numperhh char(4), cn70_hhtype char(1), cn70_region char(2), cn70_stateicp char(2), cn70_statefip char(2), cn70_countyicp char(4), cn70_countynhg char(7), cn70_metro char(1), cn70_metarea char(3), cn70_metaread char(4), cn70_city char(4), cn70_citypop char(7), cn70_sizepl char(2), cn70_urban char(1), cn70_urbpop char(7), cn70_mcdpop char(7), cn70_cntry char(3), cn70_gq char(1), cn70_gqtype char(1), cn70_gqtyped char(3), cn70_gqfunds char(2), cn70_farm char(1), cn70_nfams char(2), cn70_ncouples char(1), cn70_nmothers char(1), cn70_nfathers char(1), cn70_split char(1), cn70_splithid char(8), cn70_splitnum char(4), cn70_histidhead char(36), cn70_pernum char(4), cn70_famunit char(2), cn70_famsize char(2), cn70_momloc char(2), cn70_stepmom char(1), cn70_momrule_hist char(1), cn70_poploc char(2), cn70_steppop char(1), cn70_poprule_hist char(1), cn70_sploc char(2), cn70_sprule_hist 

Open the Database

In [27]:
conn = open_database('test')
cur = create_cursor()
set_census_schema()

Create the Census table in postgresSQL

In [28]:
create_table(table_name,column_list)


88


Commit and Close the database

In [29]:
close_database()